In [1]:
!pip install gplearn

     |████████████████████████████████| 51kB 3.7MB/s 


# Symbolic Transformer

`SymbolicTransformer` is a class from the `gplearn` package, which implements Genetic Programming in Python (drawing influence from scikit-learn).

Part of a trio:
*   `SymbolicRegressor` for regression problems.
*   `SymbolicClassifier` For binary classification.
*   `SymbolicTransformer` Automated feature engineering.

## gplearn

This package is constrained to solve symbolic regression problems.

Symbolic regression is a ML technique that tries to find a mathematical expression that most closely resembles a behavior. This is accomplished by building a population of naive random formulas to represent a relationship between known independent variables and their dependent variable targets in order to predict new data [1].

## Example: Symbolic Transformer [3]

This example is done with sklearn's Boston House-Prices Dataset for regression. The dataset has 506 samples, 13 input attributes, where attribute 14 is usually the target.

*   CRIM per capita crime rate by town
*   ZN proportion of residential land zoned for lots over 25,000 sq.ft.
*   INDUS proportion of non-retail business acres per town
*   CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
*   NOX nitric oxides concentration (parts per 10 million)
*   RM average number of rooms per dwelling
*   AGE proportion of owner-occupied units built prior to 1940
*   DIS weighted distances to five Boston employment centres
*   RAD index of accessibility to radial highways
*   TAX full-value property-tax rate per \$10,000
*   PTRATIO pupil-teacher ratio by town
*   B $1000(B_k - 0.63)^2$ where $B_k$ is the proportion of blacks by town
*   LSTAT % lower status of the population
*   MEDV Median value of owner-occupied homes in \$1000’s

First step is to load the dataset and shuffle it.


In [2]:
from gplearn.genetic import SymbolicTransformer
from sklearn.utils import check_random_state
from sklearn.datasets import load_boston
import numpy as np

rng = check_random_state(0)
boston = load_boston()
perm = rng.permutation(boston.target.size)
boston.data = boston.data[perm]
boston.target = boston.target[perm]

Only the first 300 (shuffled) samples will be used on a Ridge Regression. The remaining samples will be used to test the performance.

In [3]:
from sklearn.linear_model import Ridge
est = Ridge()
est.fit(boston.data[:300, :], boston.target[:300])
print(est.score(boston.data[300:, :], boston.target[300:]))

0.759319453049884


The benchmark to beat is $0.7593$.

Now we'll train the symbolic transformer on the same 300 samples to generate new features.

2000 individuals over 20 generations will be used. Out of them, the top 100 will be selected (`hall_of_fame`), and then the least correlated 10 will be used as the new features.

In [4]:
function_set = ["add", "sub", "mul", "div", "sqrt", "log", "abs", "neg", "inv", "max", "min"]
gp = SymbolicTransformer(generations = 20, population_size = 2000,
                         hall_of_fame = 100, n_components = 10,
                         function_set = function_set,
                         parsimony_coefficient = 0.0005,
                         max_samples = 0.9, verbose = 1,
                         random_state = 0, n_jobs=3)
gp.fit(boston.data[:300, :], boston.target[:300])

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.04         0.339876        6         0.822502         0.675124      1.14m
   1     6.91         0.593562        7         0.836993         0.602468      1.20m
   2     5.07         0.730093        8          0.84063         0.704017      1.24m
   3     5.22         0.735525        5         0.847019         0.628351      1.19m
   4     6.24         0.734679       10         0.856612         0.565138      1.12m
   5     8.23         0.721433       18          0.85677         0.728095      1.08m
   6    10.20         0.717937       14         0.875233         0.619693      1.04m
   7    11.84         0.720667       14         0.875927         0.609363     58.66s
   8    12.56         0.733019       27         0.881705         0.390121  

SymbolicTransformer(const_range=(-1.0, 1.0), feature_names=None,
                    function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                  'abs', 'neg', 'inv', 'max', 'min'],
                    generations=20, hall_of_fame=100, init_depth=(2, 6),
                    init_method='half and half', low_memory=False,
                    max_samples=0.9, metric='pearson', n_components=10,
                    n_jobs=3, p_crossover=0.9, p_hoist_mutation=0.01,
                    p_point_mutation=0.01, p_point_replace=0.05,
                    p_subtree_mutation=0.01, parsimony_coefficient=0.0005,
                    population_size=2000, random_state=0, stopping_criteria=1.0,
                    tournament_size=20, verbose=1, warm_start=False)

Now, the trained transformer will be used on the entire Boston Dataset.

In [5]:
gp_features = gp.transform(boston.data)
new_boston = np.hstack((boston.data, gp_features))

Ridge regressor is run on the first 300 samples of the transformed dataset.

In [6]:
est = Ridge()
est.fit(new_boston[:300, :], boston.target[:300])
print(est.score(new_boston[300:, :], boston.target[300:]))

0.8418372105182055


The score to beat was $0.7593$. The $R^2$ was improved by a significant margin.

# Conclusion

Genetic programming takes avantage of computing power to provide a first generation of seemingly random interactions between variables and functions, then it lets the better models pass on their knowledge to the next generation, improving their performance each subsequent generation.

A pattern may not always be found by a human at a simple glance, and bias may be included in the judgement if one is found. These random methods to predict new features eliminate these problems.

# References
*   [1] https://gplearn.readthedocs.io/en/stable/
*   [2] https://gplearn.readthedocs.io/en/stable/examples.html#symbolic-transformer
*   [3] https://github.com/trevorstephens/gplearn/blob/master/doc/gp_examples.ipynb
*   [4] https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-dataset